[983. Minimum Cost For Tickets](https://leetcode.com/problems/minimum-cost-for-tickets/description/)

The problem can be solved using dynamic programming.  
We can define a one-dimensional array `cost_on_day` of length `last_day+1`, where `last_day` is the largest day in the `days` array.  
Each element `cost_on_day[i]` of the array represents the `minimum cost to travel up to day i`. 

We can then iterate over the elements of `cost_on_day` and update them according to the minimum of the following three options:
* Buy a one-day ticket on day i, in which case the cost is `cost_on_day[i-1] + costs[0]`.  
* Buy a seven-day ticket that is valid starting from day i-7, in which case the cost is `cost_on_day[max(0,i-7)] + costs[1]`.  
* Buy a thirty-day ticket that is valid starting from day i-30, in which case the cost is `cost_on_day[max(0,i-30)] + costs[2]`.  (max(0,i-30) is used to avoid negative indices)

After iterating over the entire `cost_on_day` array, the final minimum cost will be stored in the last element `cost_on_day[-1]`.  

The idea behind the dynamic programming approach is to build up the solution incrementally,   
starting from smaller subproblems and working our way up to larger subproblems.  

In this case, we start by considering the minimum cost to travel up to day 1,   
which is simply `costs[0]` if day 1 is in the days array, or 0 if it is not.  

We then consider the minimum cost to travel up to day 2, which can be computed by considering the minimum cost  
of buying a one-day ticket on day 2, a seven-day ticket valid from day 1 to day 7, or a thirty-day ticket valid from day 1 to day 30.  
 
We can then use this information to compute the minimum cost to travel up to day 3, and so on.  

By iteratively building up the minimum cost to travel up to each day, we eventually arrive at the minimum cost  
to travel up to the largest day in the days array, which is our final solution.

In [70]:
# Problem: 983. Minimum Cost For Tickets
# Difficulty: Medium
from typing import Iterable


class Solution:
    def mincostTickets(self, days: Iterable, costs: Iterable, details=False) -> int:
        ticket_duration = [1, 7, 30]
        tickets = len(ticket_duration) # 3
        last_day = (
            max(days) + 1  
        )  # last day in days + 1 to start indexing from 1, 0 as initial state

        #  the minimum cost to travel up to day i. [0, 0, 0, 0 ...]
        cost_on_day = [0] * last_day
        ticket_used = [0] * last_day

        travel_days = set(days)  # use a set to speed up iterating

        # iterate over 'cost_on_day' elements, actually over all range of days
        for i in range(1, last_day):
            if i not in travel_days:
                # copy previous cost (zero for the first day)
                cost_on_day[i] = cost_on_day[i - 1]
                # ticket_used[i] = ticket_used[i - 1]
            else:  # choose the minima among different variants (tickets)
                cost_options = []
                for j in range(tickets):
                    # clipping index by 0, select cost backwards by ticket duration
                    # and add cost for that ticket
                    cost_options.append(
                        cost_on_day[max(0, i - ticket_duration[j])] + costs[j]
                    )
                min_cost_index = cost_options.index(min(cost_options))
                cost_on_day[i] = min(cost_options)
                
                if details:
                    ticket_used[i] = ticket_duration[min_cost_index]  # Record the index of the used ticket

                # cost_on_day[i] = min(
                #     [
                #         cost_on_day[max(0, i - ticket_duration[j])] + costs[j]
                #         for j in range(tickets)
                #     ]
                # )

        return cost_on_day[-1], ticket_used, cost_on_day

In [71]:
days = [1,4,6,     9,10,11,12,13,14,15,     16,17,18,20,21,22,  23,27,28]    # sum  44
costs = [3,13,45]
sol = Solution()
result, ticket_used, cost_on_day = sol.mincostTickets(days, costs,True)
result

44

In [72]:
ticket_used

[0,
 1,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 1,
 1,
 1,
 7,
 1,
 7,
 7,
 1,
 1,
 1,
 0,
 1,
 7,
 7,
 1,
 0,
 0,
 0,
 1,
 1]

In [73]:
len(cost_on_day)

29

In [74]:
print ('cost'+''.join(str(cost_on_day)))
# print ('days'+''.join(str(temp)))
print (''.join(str(days)))
# cost[0, 3, 3, 3, 6, 6, 9, 9, 9, 12, 15, 18, 19, 22, 22, 22, 25, 28, 31, 31, 34, 35, 35, 38, 38, 38, 38, 41, 44]
# days[0, 1, 1, 1, 4, 4, 6, 6, 6,  9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 18, 20, 21, 22, 23, 23, 23, 23, 27, 28]
#     [1,          4,    6,        9, 10, 11, 12, 13, 14, 15, 16, 17, 18,     20, 21, 22, 23,             27, 28]


cost[0, 3, 3, 3, 6, 6, 9, 9, 9, 12, 15, 18, 19, 22, 22, 22, 25, 28, 31, 31, 34, 35, 35, 38, 38, 38, 38, 41, 44]
[1, 4, 6, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 20, 21, 22, 23, 27, 28]


[Activity Selection Problem. **GREEDY approach**](https://www.techiedelight.com/activity-selection-problem/)

**Given a set of activities, along with the starting and finishing time of each activity, find the MAXIMUM number of activities performed by a single person assuming that a person can only work on a SINGLE activity at a time.**

The activity selection problem is a problem concerning selecting non-conflicting activities to perform within a given time frame, given a set of activities each marked by a `start` and `finish` time.  
A classic application of this problem is scheduling a room for multiple competing events, each having its time requirements (start and end time).

 
Let’s assume there exist $n$ activities each being represented by a start time $S[i]$ and finish time $F[j]$. Two activities $i$ and $j$ are said to be non-conflicting if  
$$S[i] = F[j]$$ 
or $$S[j] = F[i]$$

We can solve this problem by being greedy.  
Using a greedy approach will always result in an optimal solution to this problem.  

*A greedy approach is a problem-solving strategy that involves making locally optimal choices at each step in the hope of finding a global optimum solution. In other words, it involves making the best possible decision at each stage of a problem without considering the long-term consequences of those decisions.*  

The idea is to initially **sort** the activities in increasing order of their **finish** times and create a set R to store the selected activities and initialize it with the first activity.  
Then from the second activity onward, *include the activity in the activities list if the activity’s start time is greater or equal to the finish time of the last selected activity*. Repeat this for each activity involved.

In [47]:
def selectActivity(activities):
    
    result = set()  # set of selected activities INDEXES
    pointer = 0     # pointer to the last selected activity

    activities.sort(key=lambda x: x[1]) # sort activities by end time

    if len(activities) > 0:  # if there are ANY activities
        result.add(pointer)  # add first activity to the result

    for i in range(0, len(activities)): # iterate over activities

        # if start time of the current activity is greater than 
        # or equal to the end time of the last selected activity
        # then select the current activity
        start, end = activities[i][0], activities[pointer][1]
        if start >= end:  
            result.add(i)    # add current activity to the result
            pointer = i # update pointer

    return [activities[i] for i in result]  # return selected activities by INDEX

In [48]:
activities = [(1, 4), (3, 5), (0, 6), (5, 7), (3, 8), (5, 9),
        (6, 10), (8, 11), (8, 12), (2, 13), (12, 14)]

result = selectActivity(activities) 
result.sort()   # sort by start time
print (result)

[(1, 4), (5, 7), (8, 11), (12, 14)]


[Activity Selection Problem. **Dynamic programming**](https://www.techiedelight.com/activity-selection-problem-using-dynamic-programming/)

**Given a set of activities, along with the starting and finishing time of each activity, find the MAXIMUM number of activities performed by a single person assuming that a person can only work on a SINGLE activity at a time.**

The activity selection problem is a problem concerning selecting non-conflicting activities to perform within a given time frame, given a set of activities each marked by a `start` and `finish` time.  
A classic application of this problem is scheduling a room for multiple competing events, each having its time requirements (start and end time).

 
Let’s assume there exist $n$ activities each being represented by a start time $S[i]$ and finish time $F[j]$. Two activities $i$ and $j$ are said to be non-conflicting if  
$$S[i] = F[j]$$ 
or $$S[j] = F[i]$$

In the previous post, we have discussed a greedy approach for activity selection problem. This post will discuss a **dynamic programming solution** for the activity selection problem, which is nothing but a variation of the [Longest Increasing Subsequence (LIS)](https://www.techiedelight.com/longest-increasing-subsequence-using-dynamic-programming/)problem.

The idea is first to sort given activities in *increasing order of their start time*.  
Let jobs $[0…n-1]$ be the sorted array of activities.  
We can define an array $L$ such that $L[i]$ itself is an array that stores *maximum non-conflicting jobs  
ending at $i'th$ job*.  
Therefore, $L[i]$ can be recursively written as:  

```L[i] = jobs[i] + { max(L[j]), where j < i and jobs[j].end < jobs[i].start } ```

     = jobs[i], if there is no such j




In [309]:
def findNonConflictingJobsLength(jobs, print_=False):
 
    
    jobs.sort(key=lambda x: x[0]) # Sort the jobs according to INCREASING order of their START time
 
    # L[i] stores the maximum count of non-conflicting jobs at given INDEX i
    # the number of jobs that can be done before job i
    L = [0] * len(jobs) # [0, 0, 0, 0, 0, ...]
    if print_:
        print('job[i]start AFTER job[j]end i , j   L')

    for i in range(len(jobs)): # consider each `i`th job
               
        for j in range(i):  # consider each `j` less than `i`
            if print_:
                print(f'{jobs[i]}  {jobs[j]}        job{i:>1},iter{j:>1} {L}')

            # Look for previous job `j` in LIST
            # where `j` job finish before `jobs[i].start` 
            start, finish = jobs[i][0], jobs[j][1]
            if L[i] < L[j] and start > finish:
                # if L[j] > 1 then on j position there is already more 
                # than 1 job and now we managed to add one more job
                if print_:
                    print(f'YES! jobs[{i}]start after jobs[{j}]end ------------   {L}')
                L[i] = L[j] # update L[i] with L[j] 

        L[i] = L[i] + 1 # increment current value of L[i] 

        print(f'-----------next {jobs[i]}   FOUND {L[i]} jobs  L = {L}')

    # return the maximum job length in the list
    return max(L)

In [310]:
jobs = [(0, 6), (1, 4), (2, 13), (3, 5), (3, 8), (5, 7), (5, 9), (6, 10), (8, 11), (8, 12), (12, 14)]

result = findNonConflictingJobsLength(jobs, print_ = False) 
# result.sort()   # sort by start time
print (result)

-----------next (0, 6)   FOUND 1 jobs  L = [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
-----------next (1, 4)   FOUND 1 jobs  L = [1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0]
-----------next (2, 13)   FOUND 1 jobs  L = [1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0]
-----------next (3, 5)   FOUND 1 jobs  L = [1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0]
-----------next (3, 8)   FOUND 1 jobs  L = [1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0]
-----------next (5, 7)   FOUND 2 jobs  L = [1, 1, 1, 1, 1, 2, 0, 0, 0, 0, 0]
-----------next (5, 9)   FOUND 2 jobs  L = [1, 1, 1, 1, 1, 2, 2, 0, 0, 0, 0]
-----------next (6, 10)   FOUND 2 jobs  L = [1, 1, 1, 1, 1, 2, 2, 2, 0, 0, 0]
-----------next (8, 11)   FOUND 3 jobs  L = [1, 1, 1, 1, 1, 2, 2, 2, 3, 0, 0]
-----------next (8, 12)   FOUND 3 jobs  L = [1, 1, 1, 1, 1, 2, 2, 2, 3, 3, 0]
-----------next (12, 14)   FOUND 4 jobs  L = [1, 1, 1, 1, 1, 2, 2, 2, 3, 3, 4]
4


In [ ]:
# Find the maximum number of non-conflicting jobs that can be performed
# by a single person
def findNonConflictingJobs(jobs):
 
    # sort the jobs according to increasing order of their start time
    jobs.sort(key=lambda x: x[0])
 
    # `L[i]` stores the maximum non-conflicting jobs that end at i'th job
    # `L[i]` is a list of jobs (tuples)
    L = [[] for _ in range(len(jobs))]
 
    for i in range(len(jobs)):
        # consider each `j` less than `i`
        for j in range(i):
            # L[i] = max(L[j]), where `jobs[j].finish` is less than `jobs[i].start`
            start, finish = (jobs[i][0], jobs[j][1])
            if finish < start and len(L[i]) < len(L[j]):
                L[i] = L[j].copy()
 
        # `L[i]` ends at i'th job
        L[i].append(jobs[i])
        print(f'L[{i}]= {L[i]} ')
        
           # find the list having a maximum size
    # print(L)
    max = []
    for pair in L:
        if len(max) < len(pair):
            max = pair
 
    # print maximum non-conflicting jobs
    print(max)

In [ ]:
jobs = [(1, 4), (3, 5), (0, 6), (5, 7),
        (3, 8), (5, 9), (6, 10), (8, 11),
        (8, 12), (2, 13), (12, 14)]

findNonConflictingJobs(jobs)

L[0]= [(0, 6)] 
L[1]= [(1, 4)] 
L[2]= [(2, 13)] 
L[3]= [(3, 5)] 
L[4]= [(3, 8)] 
L[5]= [(1, 4), (5, 7)] 
L[6]= [(1, 4), (5, 9)] 
L[7]= [(1, 4), (6, 10)] 
L[8]= [(1, 4), (5, 7), (8, 11)] 
L[9]= [(1, 4), (5, 7), (8, 12)] 
L[10]= [(1, 4), (5, 7), (8, 11), (12, 14)] 
[(1, 4), (5, 7), (8, 11), (12, 14)]


### Longest Increasing Subsequence using Dynamic Programming

The longest increasing subsequence problem is to find a subsequence of a given sequence in which the subsequence’s elements are in sorted order, lowest to highest, and in which the subsequence is as long as possible. This subsequence is not necessarily contiguous or unique.

Please note that the problem specifically targets subsequences that need not be contiguous, i.e., subsequences are not required to occupy consecutive positions within the original sequences.

For example, the longest increasing subsequence of [0, 8, 4, 12, 2, 10, 6, 14, 1, 9, 5, 13, 3, 11, 7, 15] is [0, 2, 6, 9, 11, 15].

This subsequence has length 6; the input sequence has no 7–member increasing subsequences. The longest increasing subsequence in this example is not unique.

For instance, [0, 4, 6, 9, 11, 15] or [0, 4, 6, 9, 13, 15] are other increasing subsequences of equal length in the same input sequence.

In [304]:
# Iterative function to find the length of the longest increasing subsequence
# of a given list
def LIS(arr):
 
    # base case
    if not arr:
        return 0
 
    # list to store subproblem solutions. `L[i]` stores the length
    # of the longest increasing subsequence that ends with `arr[i]`
    L = [0] * len(arr)
 
    # the longest increasing subsequence ending at `arr[0]` has length 1
    L[0] = 1
 
    # start from the second element in the list
    for i in range(1, len(arr)):
        # do for each element in sublist `arr[0…i-1]`
        for j in range(i):
            # find the longest increasing subsequence that ends with `arr[j]`
            # where `arr[j]` is less than the current element `arr[i]`
            if arr[j] < arr[i] and L[j] > L[i]:
                L[i] = L[j]
                print(f'YES! {arr[j]:>3}   < {arr[i]:>3}  ------------  {L}')
 
        # include `arr[i]` in LIS
        L[i] = L[i] + 1
        print(f'---next {arr[i]:>3}   FOUND {L[i]} subs  L = {L}')
 
    # return longest increasing subsequence (having maximum length)
    return max(L)
 
 
if __name__ == '__main__':
 
    arr = [0, 8, 4, 12, 2, 10, 6, 14, 1, 9, 5, 13, 3, 11, 7, 15]
    print('The length of the LIS is', LIS(arr))

YES!   0   <   8  ------------  [1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
---next   8   FOUND 2 subs  L = [1, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
YES!   0   <   4  ------------  [1, 2, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
---next   4   FOUND 2 subs  L = [1, 2, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
YES!   0   <  12  ------------  [1, 2, 2, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
YES!   8   <  12  ------------  [1, 2, 2, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
---next  12   FOUND 3 subs  L = [1, 2, 2, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
YES!   0   <   2  ------------  [1, 2, 2, 3, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
---next   2   FOUND 2 subs  L = [1, 2, 2, 3, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
YES!   0   <  10  ------------  [1, 2, 2, 3, 2, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
YES!   8   <  10  ------------  [1, 2, 2, 3, 2, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
---next  10   FOUND 3 subs  L = [1, 2, 2, 3, 2, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
YES!   0   <   6  ----------

### Binary Search of a sorted list

In [314]:
# Function to determine if a `target` exists in the sorted list `nums`
# or not using a binary search algorithm
def binarySearch(nums, target):
 
    # search space is nums[left…right]
    (left, right) = (0, len(nums) - 1)
 
    # loop till the search space is exhausted
    while left <= right:
 
        # find the mid-value in the search space and
        # compares it with the target
 
        mid = (left + right) // 2
 
        # overflow can happen. Use:
        # mid = left + (right - left) / 2
        # mid = right - (right - left) // 2
 
        # target is found
        if target == nums[mid]:
            return mid
 
        # discard all elements in the right search space,
        # including the middle element
        elif target < nums[mid]:
            right = mid - 1
 
        # discard all elements in the left search space,
        # including the middle element
        else:
            left = mid + 1
 
    # `target` doesn't exist in the list
    return -1
 
 
if __name__ == '__main__':
 
    nums = [2, 5, 6, 8, 9, 10]
    target = 5
 
    index = binarySearch(nums, target)
 
    if index != -1:
        print('Element found at index', index)
    else:
        print('Element found not in the list')
 

Element found at index 1
